# Function 3: Updating SQL Database with pyODBC

Write a function which:
- Connects and updates your SQL database. 

Function Specifications:
- The function should take in a pandas dataframe created in Function 2. 
- Connect to your SQL database.
- Update the table you store your tweets in.
- Not return any output.

In [0]:
def pyodbc_twitter(connection, df, twitter_table):

  ''' Update 'Tweets' table in our SQL database

      Args: 'connection', 'df', 'twitter_table'

      Returns: None, only updates the 'Twitter' table
  '''

  ### Code Here

  return None 